## Installing Libraries

In [76]:
!pip install langchain
!pip install openai
!pip install streamlit
!pip install langchain
!pip install openai
!pip install utils
!pip install PyPDF2
!pip install jsonlib
!pip install traceback2
!pip install pyngrok
!pip install python-docx
!pip install Python-IO
!pip install XlsxWriter

  Using cached jsonlib-1.6.1.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for jsonlib
  Running setup.py clean for jsonlib
Failed to build jsonlib
ERROR: Could not build wheels for jsonlib, which is required to install pyproject.toml-based projects


## StreamLit APP Code


In [77]:

%%writefile app.py

#Importing Libs
import json
import os
import io
import streamlit as st
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import pandas as pd
import traceback
import PyPDF2
import docx


#To extract contents from given .pdf or.txt file
def parse_file(file):
    if file.name.endswith(".pdf"):
        try:
            pdf_reader=PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return text
        except PyPDF2.PdfReadError:
            raise Exception("Error reading the PDF file.")

    elif file.name.endswith(".txt"):
        return file.read().decode("utf-8")

    elif file.name.endswith(".docx"):
        document = docx.Document(file)
        paragraphs = [p.text for p in document.paragraphs]
        text = '\n'.join(paragraphs)
        return text

    else:
        raise Exception(
            "Unsupported File Format. Only PDF and .Txt Files are supported"
        )

#final output format
RESPONSE_JSON = {
    "1": {
        "no": "1",
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "no": "2",
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "no": "3",
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

# converting the final output to table format
def get_table_data(quiz_str):
    try:
        # convert the quiz from a str to dict
        quiz_dict = json.loads(quiz_str)
        quiz_table_data = []
        # Iterate over the quiz dictionary and extract the required information
        for key, value in quiz_dict.items():
            mcq = value["mcq"]
            options = " | ".join(
                [
                    f"{option}: {option_value}"
                    for option, option_value in value["options"].items()
                ]
            )
            correct = value["correct"]
            quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
        return quiz_table_data
    except Exception as e:
        traceback.print_exception(type(e), e, e.__traceback__)
        return False


# setting LLM Key - OpenApi
os.environ["OPENAI_API_KEY"] =  "Your-OPENAPI-KEY"

# initializing the LLM model
llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0)


#prompt template 1 as input for LLM
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for grade {grade} students in {tone} tone with Questions type as {type}.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "grade", "tone","type", "response_json"],
    template=template

)

#creating Chain-1
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)



#prompt template 2 as input for LLM
template="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {grade} grade students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis.
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update teh quiz questions which needs to be changed  and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Critique from an expert English Writer of the above quiz:
"""
#creating Chain-2
quiz_evaluation_prompt=PromptTemplate(input_variables=["grade", "quiz"], template=template)
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


# This is an Overall Chain where we run the two chains in Sequence
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "grade", "tone","type", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

st.title("LLM Chatbot to Create MCQs 🦜")


#Create a form using st.form

with st.form("user_inputs"):
    #File Upload
    uploaded_file=st.file_uploader("Uplaod a PDF or txt or docx file")

    #Input Fields
    mcq_count=st.number_input("No. of MCQs", min_value=3, max_value=50)

    #grade
    grade=st.number_input("Insert Grade", min_value=1, max_value=10)

    # Quiz Tone
    tone=st.text_input("Insert Quiz Tone", max_chars=100, placeholder="simple")

    # Question Type
    type=st.text_input("Question Type", max_chars=100, placeholder="True/False etc")

    #Add Button
    button=st.form_submit_button("Create MCQs")

    # Check if the button is clicked and all fields have input

    if button and uploaded_file is not None and mcq_count and grade and tone:
        with st.spinner("loading..."):
            try:
                text=parse_file(uploaded_file)
                #Count tokens and the cost of API call
                with get_openai_callback() as cb:
                    response=generate_evaluate_chain(
                        {
                        "text": text,
                        "number": mcq_count,
                        "grade":grade,
                        "tone": tone,
                        "type": type,
                        "response_json": json.dumps(RESPONSE_JSON)
                            }
                    )
                #st.write(response)


            except Exception as e:
                traceback.print_exception(type(e), e, e.__traceback__)
                st.error("Error")

            else:
                print(f"Total Tokens:{cb.total_tokens}")
                print(f"Prompt Tokens:{cb.prompt_tokens}")
                print(f"Completion Tokens:{cb.completion_tokens}")
                print(f"Total Cost:{cb.total_cost}")
                st.text_area(label="Cost in Dollars", value=cb.total_cost)
                if isinstance(response, dict):
                    #Extract the quiz data from the response
                    quiz=response.get("quiz", None)
                    if quiz is not None:
                        table_data=get_table_data(quiz)
                        if table_data is not None:
                            df=pd.DataFrame(table_data)
                            df.index=df.index+1
                            st.table(df)
                            #Display the review in atext box as well
                            st.text_area(label="Review", value=response["review"])
                        else:
                            st.error("Error in the table data")

                else:
                    st.write(response)




Overwriting app.py


## Get Public IP to Run App

In [78]:
!wget -q -O - ipv4.icanhazip.com

34.141.215.27


## Run Streamlit app in WEB using Local Tunnel

In [ ]:
!npm install localtunnel
!streamlit run /content/app.py &>/dev/null&
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.932s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

npx: installed 22 in 4.872s
your url is: https://shaggy-chefs-spend.loca.lt
